## Biblioteki

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from openpyxl import load_workbook

## Pobór danych 

In [36]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Długość ścieżki

In [37]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Algorytm genetyczny (GA)

In [38]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene

    return child

def crossover_pmx(parent1, parent2):
    size = len(parent1)
    start, end = 2, 6
    child = [None] * size

    # Skopiuj segment z 1-ego rodzica do dziecka
    child[start:end] = parent1[start:end]

    # Wypełnij pozostałe pozycje z 2-ego rodzica
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in child:
                gene = parent2[parent1.index(gene)]
            child[i] = gene

    return child

def selection_tournament(population, scores, k=3):
    # Wybierz k losowych osobników z populacji
    selected = random.sample(range(len(population)), k)
    # Posortuj wybrane osobniki według ich wyników
    selected = sorted(selected, key=lambda x: scores[x])
    # Zwróć najlepszego osobnika
    return population[selected[0]]

def selection_roulette(population, scores):
    total_score = sum(scores)
    
    # Oblicz prawdopodobieństwa
    probabilities = [score / total_score for score in scores]

    # Oblicz skumulowane prawdopodobieństwa
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    # Wybierz losową liczbę między 0 a 1
    pick = random.uniform(0, 1)

    # Wybierz osobnika na podstawie losowej liczby
    for i, cumulative_probability in enumerate(cumulative_probabilities):
        if pick <= cumulative_probability:
            return population[i]


def mutate_swap(individual):
    # Zamień dwa losowe geny miejscami
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]

def mutate_inversion(individual):
    # Odwróć kolejność genów między dwoma losowymi punktami
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])

def mutate_insertion(individual):
    # Wstaw losowy gen w inne miejsce
    i, j = random.sample(range(len(individual)), 2)
    gene = individual.pop(i)
    individual.insert(j, gene)


def genetic_algorithm(distance_matrix, population_size=10, generations=5000, crossover_rate=0.6, mutation_rate=0.15, selection_method='tournament', crossover_method='order', mutation_method='swap'):
    start_time = time.time()

    n = len(distance_matrix)
    # Inicjalizuj populację losowymi permutacjami
    population = [random.sample(range(n), n) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')
    best_time = 0

    for generation in range(1, generations + 1):
        # Oblicz wyniki dla populacji
        scores = [calculate_path_cost(distance_matrix, individual) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            # Wybierz rodziców
            if selection_method == 'tournament':
                parent1 = selection_tournament(population, scores)
                parent2 = selection_tournament(population, scores)
            elif selection_method == 'roulette':
                parent1 = selection_roulette(population, scores)
                parent2 = selection_roulette(population, scores)

            # Krzyżowanie rodziców
            if random.random() < crossover_rate:
                if crossover_method == 'order':
                    child1 = crossover_order(parent1, parent2)
                    child2 = crossover_order(parent2, parent1)
                elif crossover_method == 'pmx':
                    child1 = crossover_pmx(parent1, parent2)
                    child2 = crossover_pmx(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]

            # Mutacja dzieci
            if random.random() < mutation_rate:
                if mutation_method == 'swap':
                    mutate_swap(child1)
                    mutate_swap(child2)
                elif mutation_method == 'inversion':
                    mutate_inversion(child1)
                    mutate_inversion(child2)
                elif mutation_method == 'insertion':
                    mutate_insertion(child1)
                    mutate_insertion(child2)

            new_population.extend([child1, child2])

        population = new_population

        # Znajdź najlepszego osobnika w populacji
        for individual in population:
            cost = calculate_path_cost(distance_matrix, individual)
            if cost < best_cost:
                best_time = time.time() - start_time
                best_solution = individual
                best_cost = cost

        # Wyświetl informacje co 100 iteracji lub na końcu
        if (generation % 100 == 0) or (generation == generations):
            elapsed_time = time.time() - start_time  # Oblicz czas, który upłynął
            print(f"Iteration {generation}, Best Cost: {best_cost: .2f}, Elapsed Time: {elapsed_time:.2f} seconds")

    best_solution = [x + 1 for x in best_solution]  # Zmień indeksy na numery miast
    return best_solution, best_cost, best_time

### Generowanie wyników

Za podstawowe dane przyjmujemy:

* population_size = 10 

* generations = 5000

* crossover_rate = 0.6

* mutation_rate = 0.15

* selection_method = `tournament`
    
* crossover_method = `order`
    
* mutation_method = `swap`

In [39]:
# Dane_TSP_48
best_path_1, best_cost_1, best_time_1 = genetic_algorithm(data1)
# Dane_TSP_76
best_path_2, best_cost_2, best_time_2 = genetic_algorithm(data2)
# Dane_TSP_127
best_path_3, best_cost_3, best_time_3 = genetic_algorithm(data3)

Iteration 100, Best Cost:  28935.00, Elapsed Time: 0.05 seconds
Iteration 200, Best Cost:  23319.00, Elapsed Time: 0.09 seconds
Iteration 300, Best Cost:  19787.00, Elapsed Time: 0.13 seconds
Iteration 400, Best Cost:  18008.00, Elapsed Time: 0.17 seconds
Iteration 500, Best Cost:  17662.00, Elapsed Time: 0.21 seconds
Iteration 600, Best Cost:  17533.00, Elapsed Time: 0.24 seconds
Iteration 700, Best Cost:  17121.00, Elapsed Time: 0.28 seconds
Iteration 800, Best Cost:  16501.00, Elapsed Time: 0.32 seconds
Iteration 900, Best Cost:  16497.00, Elapsed Time: 0.36 seconds
Iteration 1000, Best Cost:  16020.00, Elapsed Time: 0.40 seconds
Iteration 1100, Best Cost:  15784.00, Elapsed Time: 0.45 seconds
Iteration 1200, Best Cost:  15738.00, Elapsed Time: 0.49 seconds
Iteration 1300, Best Cost:  15738.00, Elapsed Time: 0.54 seconds
Iteration 1400, Best Cost:  15738.00, Elapsed Time: 0.59 seconds
Iteration 1500, Best Cost:  15659.00, Elapsed Time: 0.63 seconds
Iteration 1600, Best Cost:  15659.

In [40]:
W1 =  {
    "Długość ścieżki": [best_cost_1, best_cost_2, best_cost_3],
    "Ścieżka": [best_path_1, best_path_2, best_path_3],
    "Czas": [best_time_1, best_time_2, best_time_3]
}

basic = pd.DataFrame(data = W1)

with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

#### Badanie wpływu poszczególnych parametrów

Badanie wpływu parametru *population_size*

In [41]:
def test_population_size(data, dataset_name, num_repeats=10):
    results = []
    population_sizes = [5, 10, 15, 20]
    for population_size in population_sizes:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, population_size=population_size)
            results.append({"PARAMETR": population_size, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_population_size_1 = test_population_size(data1, "DATA1")
df_population_size_2 = test_population_size(data2, "DATA2")
df_population_size_3 = test_population_size(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
population_size = pd.concat([df_population_size_1, df_population_size_2, df_population_size_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    population_size.to_excel(writer, sheet_name='population_size', index=False)

Iteration 100, Best Cost:  36362.00, Elapsed Time: 0.03 seconds
Iteration 200, Best Cost:  33247.00, Elapsed Time: 0.05 seconds
Iteration 300, Best Cost:  31405.00, Elapsed Time: 0.07 seconds
Iteration 400, Best Cost:  30045.00, Elapsed Time: 0.09 seconds
Iteration 500, Best Cost:  29480.00, Elapsed Time: 0.11 seconds
Iteration 600, Best Cost:  28154.00, Elapsed Time: 0.14 seconds
Iteration 700, Best Cost:  27278.00, Elapsed Time: 0.16 seconds
Iteration 800, Best Cost:  25226.00, Elapsed Time: 0.19 seconds
Iteration 900, Best Cost:  24895.00, Elapsed Time: 0.22 seconds
Iteration 1000, Best Cost:  23988.00, Elapsed Time: 0.25 seconds
Iteration 1100, Best Cost:  22588.00, Elapsed Time: 0.27 seconds
Iteration 1200, Best Cost:  22588.00, Elapsed Time: 0.30 seconds
Iteration 1300, Best Cost:  22588.00, Elapsed Time: 0.33 seconds
Iteration 1400, Best Cost:  22192.00, Elapsed Time: 0.35 seconds
Iteration 1500, Best Cost:  21070.00, Elapsed Time: 0.38 seconds
Iteration 1600, Best Cost:  21070.

Badanie wpływu parametru *mutation_rate*

In [42]:
def test_mutation_rate(data, dataset_name, num_repeats=10):
    results = []
    mutation_rates = [0.01, 0.05, 0.1, 0.2]
    for mutation_rate in mutation_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_rate=mutation_rate)
            results.append({"PARAMETR": mutation_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_rate_1 = test_mutation_rate(data1, "DATA1")
df_mutation_rate_2 = test_mutation_rate(data2, "DATA2")
df_mutation_rate_3 = test_mutation_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_rate = pd.concat([df_mutation_rate_1, df_mutation_rate_2, df_mutation_rate_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    mutation_rate.to_excel(writer, sheet_name='mutation_rate', index=False)

Iteration 100, Best Cost:  38154.00, Elapsed Time: 0.06 seconds
Iteration 200, Best Cost:  37238.00, Elapsed Time: 0.12 seconds
Iteration 300, Best Cost:  36857.00, Elapsed Time: 0.17 seconds
Iteration 400, Best Cost:  36857.00, Elapsed Time: 0.23 seconds
Iteration 500, Best Cost:  36489.00, Elapsed Time: 0.28 seconds
Iteration 600, Best Cost:  36489.00, Elapsed Time: 0.33 seconds
Iteration 700, Best Cost:  36489.00, Elapsed Time: 0.39 seconds
Iteration 800, Best Cost:  36489.00, Elapsed Time: 0.45 seconds
Iteration 900, Best Cost:  35376.00, Elapsed Time: 0.50 seconds
Iteration 1000, Best Cost:  32443.00, Elapsed Time: 0.55 seconds
Iteration 1100, Best Cost:  32321.00, Elapsed Time: 0.60 seconds
Iteration 1200, Best Cost:  29229.00, Elapsed Time: 0.66 seconds
Iteration 1300, Best Cost:  29229.00, Elapsed Time: 0.71 seconds
Iteration 1400, Best Cost:  28713.00, Elapsed Time: 0.76 seconds
Iteration 1500, Best Cost:  26518.00, Elapsed Time: 0.81 seconds
Iteration 1600, Best Cost:  25388.

Badanie wpływu parametru *crossover_rate*

In [43]:
def test_crossover_rate(data, dataset_name, num_repeats=10):
    results = []
    crossover_rates = [0.6, 0.7, 0.8, 0.9]
    for crossover_rate in crossover_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_rate=crossover_rate)
            results.append({"PARAMETR": crossover_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_rate_1 = test_crossover_rate(data1, "DATA1")
df_crossover_rate_2 = test_crossover_rate(data2, "DATA2")
df_crossover_rate_3 = test_crossover_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_rate = pd.concat([df_crossover_rate_1, df_crossover_rate_2, df_crossover_rate_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    crossover_rate.to_excel(writer, sheet_name='crossover_rate', index=False)

Iteration 100, Best Cost:  27764.00, Elapsed Time: 0.07 seconds
Iteration 200, Best Cost:  25373.00, Elapsed Time: 0.14 seconds
Iteration 300, Best Cost:  24357.00, Elapsed Time: 0.21 seconds
Iteration 400, Best Cost:  22682.00, Elapsed Time: 0.27 seconds
Iteration 500, Best Cost:  21033.00, Elapsed Time: 0.33 seconds
Iteration 600, Best Cost:  20517.00, Elapsed Time: 0.40 seconds
Iteration 700, Best Cost:  19941.00, Elapsed Time: 0.45 seconds
Iteration 800, Best Cost:  19488.00, Elapsed Time: 0.51 seconds
Iteration 900, Best Cost:  18378.00, Elapsed Time: 0.57 seconds
Iteration 1000, Best Cost:  16991.00, Elapsed Time: 0.63 seconds
Iteration 1100, Best Cost:  16846.00, Elapsed Time: 0.69 seconds
Iteration 1200, Best Cost:  16540.00, Elapsed Time: 0.76 seconds
Iteration 1300, Best Cost:  16527.00, Elapsed Time: 0.82 seconds
Iteration 1400, Best Cost:  16506.00, Elapsed Time: 0.88 seconds
Iteration 1500, Best Cost:  16334.00, Elapsed Time: 0.94 seconds
Iteration 1600, Best Cost:  16095.

Badanie wpływu parametru *generations*

In [44]:
def test_generations(data, dataset_name, num_repeats=10):
    results = []
    generations_list = [5000, 7500, 10000, 15000]
    for generations in generations_list:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, generations=generations)
            results.append({"PARAMETR": generations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_generations_1 = test_generations(data1, "DATA1")
df_generations_2 = test_generations(data2, "DATA2")
df_generations_3 = test_generations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
generations = pd.concat([df_generations_1, df_generations_2, df_generations_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    generations.to_excel(writer, sheet_name='generations', index=False)

Iteration 100, Best Cost:  31633.00, Elapsed Time: 0.05 seconds
Iteration 200, Best Cost:  29128.00, Elapsed Time: 0.10 seconds
Iteration 300, Best Cost:  25878.00, Elapsed Time: 0.15 seconds
Iteration 400, Best Cost:  25435.00, Elapsed Time: 0.20 seconds
Iteration 500, Best Cost:  23380.00, Elapsed Time: 0.25 seconds
Iteration 600, Best Cost:  21987.00, Elapsed Time: 0.30 seconds
Iteration 700, Best Cost:  20902.00, Elapsed Time: 0.37 seconds
Iteration 800, Best Cost:  20671.00, Elapsed Time: 0.42 seconds
Iteration 900, Best Cost:  20182.00, Elapsed Time: 0.47 seconds
Iteration 1000, Best Cost:  19942.00, Elapsed Time: 0.53 seconds
Iteration 1100, Best Cost:  19928.00, Elapsed Time: 0.59 seconds
Iteration 1200, Best Cost:  19221.00, Elapsed Time: 0.65 seconds
Iteration 1300, Best Cost:  18479.00, Elapsed Time: 0.72 seconds
Iteration 1400, Best Cost:  18284.00, Elapsed Time: 0.79 seconds
Iteration 1500, Best Cost:  17568.00, Elapsed Time: 0.85 seconds
Iteration 1600, Best Cost:  17331.

Badanie wpływu parametru *selection_method*

In [45]:
def test_selection_method(data, dataset_name, num_repeats=10):
    results = []
    selection_methods = ['roulette', 'tournament']
    for selection_method in selection_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, selection_method=selection_method)
            results.append({"PARAMETR": selection_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_selection_method_1 = test_selection_method(data1, "DATA1")
df_selection_method_2 = test_selection_method(data2, "DATA2")
df_selection_method_3 = test_selection_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
selection_method = pd.concat([df_selection_method_1, df_selection_method_2, df_selection_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    selection_method.to_excel(writer, sheet_name='selection_method', index=False)

Iteration 100, Best Cost:  44526.00, Elapsed Time: 0.10 seconds
Iteration 200, Best Cost:  44526.00, Elapsed Time: 0.19 seconds
Iteration 300, Best Cost:  44526.00, Elapsed Time: 0.29 seconds
Iteration 400, Best Cost:  44526.00, Elapsed Time: 0.39 seconds
Iteration 500, Best Cost:  44526.00, Elapsed Time: 0.49 seconds
Iteration 600, Best Cost:  40749.00, Elapsed Time: 0.59 seconds
Iteration 700, Best Cost:  40749.00, Elapsed Time: 0.70 seconds
Iteration 800, Best Cost:  40749.00, Elapsed Time: 0.82 seconds
Iteration 900, Best Cost:  40749.00, Elapsed Time: 0.94 seconds
Iteration 1000, Best Cost:  40749.00, Elapsed Time: 1.08 seconds
Iteration 1100, Best Cost:  38983.00, Elapsed Time: 1.19 seconds
Iteration 1200, Best Cost:  38983.00, Elapsed Time: 1.31 seconds
Iteration 1300, Best Cost:  38983.00, Elapsed Time: 1.42 seconds
Iteration 1400, Best Cost:  38983.00, Elapsed Time: 1.53 seconds
Iteration 1500, Best Cost:  38983.00, Elapsed Time: 1.62 seconds
Iteration 1600, Best Cost:  38983.

Badanie wpływu parametru *crossover_method*

In [46]:
def test_crossover_method(data, dataset_name, num_repeats=10):
    results = []
    crossover_methods = ['order', 'pmx']
    for crossover_method in crossover_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_method=crossover_method)
            results.append({"PARAMETR": crossover_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_method_1 = test_crossover_method(data1, "DATA1")
df_crossover_method_2 = test_crossover_method(data2, "DATA2")
df_crossover_method_3 = test_crossover_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_method = pd.concat([df_crossover_method_1, df_crossover_method_2, df_crossover_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    crossover_method.to_excel(writer, sheet_name='crossover_method', index=False)

Iteration 100, Best Cost:  29118.00, Elapsed Time: 0.06 seconds
Iteration 200, Best Cost:  27653.00, Elapsed Time: 0.11 seconds
Iteration 300, Best Cost:  26639.00, Elapsed Time: 0.17 seconds
Iteration 400, Best Cost:  25713.00, Elapsed Time: 0.23 seconds
Iteration 500, Best Cost:  23797.00, Elapsed Time: 0.30 seconds
Iteration 600, Best Cost:  23322.00, Elapsed Time: 0.36 seconds
Iteration 700, Best Cost:  22082.00, Elapsed Time: 0.43 seconds
Iteration 800, Best Cost:  21911.00, Elapsed Time: 0.48 seconds
Iteration 900, Best Cost:  21230.00, Elapsed Time: 0.54 seconds
Iteration 1000, Best Cost:  21141.00, Elapsed Time: 0.60 seconds
Iteration 1100, Best Cost:  21012.00, Elapsed Time: 0.66 seconds
Iteration 1200, Best Cost:  21012.00, Elapsed Time: 0.72 seconds
Iteration 1300, Best Cost:  20850.00, Elapsed Time: 0.77 seconds
Iteration 1400, Best Cost:  20850.00, Elapsed Time: 0.82 seconds
Iteration 1500, Best Cost:  20643.00, Elapsed Time: 0.88 seconds
Iteration 1600, Best Cost:  20643.

Badanie wpływu parametru *mutation_method*

In [47]:
def test_mutation_method(data, dataset_name, num_repeats=10):
    results = []
    mutation_methods = ['insertion', 'swap', 'inversion']
    for mutation_method in mutation_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_method=mutation_method)
            results.append({"PARAMETR": mutation_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_method_1 = test_mutation_method(data1, "DATA1")
df_mutation_method_2 = test_mutation_method(data2, "DATA2")
df_mutation_method_3 = test_mutation_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_method = pd.concat([df_mutation_method_1, df_mutation_method_2, df_mutation_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    mutation_method.to_excel(writer, sheet_name='mutation_method', index=False)

Iteration 100, Best Cost:  28783.00, Elapsed Time: 0.04 seconds
Iteration 200, Best Cost:  25021.00, Elapsed Time: 0.09 seconds
Iteration 300, Best Cost:  22566.00, Elapsed Time: 0.17 seconds
Iteration 400, Best Cost:  21640.00, Elapsed Time: 0.22 seconds
Iteration 500, Best Cost:  21528.00, Elapsed Time: 0.27 seconds
Iteration 600, Best Cost:  19826.00, Elapsed Time: 0.33 seconds
Iteration 700, Best Cost:  18214.00, Elapsed Time: 0.38 seconds
Iteration 800, Best Cost:  17710.00, Elapsed Time: 0.44 seconds
Iteration 900, Best Cost:  16900.00, Elapsed Time: 0.49 seconds
Iteration 1000, Best Cost:  16859.00, Elapsed Time: 0.54 seconds
Iteration 1100, Best Cost:  16482.00, Elapsed Time: 0.60 seconds
Iteration 1200, Best Cost:  16139.00, Elapsed Time: 0.65 seconds
Iteration 1300, Best Cost:  15897.00, Elapsed Time: 0.70 seconds
Iteration 1400, Best Cost:  15897.00, Elapsed Time: 0.75 seconds
Iteration 1500, Best Cost:  15794.00, Elapsed Time: 0.80 seconds
Iteration 1600, Best Cost:  15590.